# Smart Classroom Data Evaluation

## Setup

In [1]:
import psycopg2
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

db_hostname = config['DATABASE']['HOSTNAME']
db_database = config['DATABASE']['DATABASE']
db_username = config['DATABASE']['USERNAME']
db_password = config['DATABASE']['PASSWORD']

## Test connection

In [3]:
conn = psycopg2.connect(
    host=db_hostname,
    database=db_database,
    user=db_username,
    password=db_password)


cur = conn.cursor()
        
print('PostgreSQL database version:')
cur.execute('SELECT version()')
print(cur.fetchone())

cur.close()
conn.close()


PostgreSQL database version:
('PostgreSQL 14.2 on x86_64-pc-linux-musl, compiled by gcc (Alpine 10.3.1_git20211027) 10.3.1 20211027, 64-bit',)
